# En este notebook se analiza el comportamiento de los diferentes grupos de postulantes de a cuerdo a sus edades. 
## Para esto se toman 3 grupos de postulantes: 
### - Los menores de 23 años (18 - 22 años).
### - Los que se encuentran entre 23 años y 34 años inclusive (23 - 34 años).
### - Los mayores de 35 años.

In [1]:
# Se importan las librerías necesarias.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime

pd.options.mode.chained_assignment = None

In [2]:
# Se leen los dataframes.
postulantesEducacion = pd.read_csv('../Data/fiuba_1_postulantes_educacion.csv')
postulantesGeneroYEdad = pd.read_csv('../Data/fiuba_2_postulantes_genero_y_edad.csv')
oportunidadVistas = pd.read_csv('../Data/fiuba_3_vistas.csv')
oportunidadPostulaciones = pd.read_csv('../Data/fiuba_4_postulaciones.csv')
avisosOnline = pd.read_csv('../Data/fiuba_5_avisos_online.csv')
avisosDetalle = pd.read_csv('../Data/fiuba_6_avisos_detalle.csv')

In [3]:
# Elimino los postulantes cuyos ids o fechas de nacimiento son nulas.

# Elimino las filas que no tengan id del postulante.
postulantesEducacion.dropna(subset = ['idpostulante'], inplace = True)
postulantesGeneroYEdad.dropna(subset = ['idpostulante'], inplace = True)

# Creo un nuevo dataframe con los postulantes que tengan fecha de nacimiento.
postulantesConEdad = postulantesGeneroYEdad.dropna(subset=['fechanacimiento'])

In [4]:
# Creo una nueva columna con la edad del postulante. Descarto los postulantes cuyas edades no pueden ser averiguadas.

# Paso la columna fecha de nacimiento a un datetime.
postulantesConEdad['fechanacimiento'] = pd.to_datetime(postulantesConEdad['fechanacimiento'], errors='coerce')

# Creo la columna edad como diferencia entre la fecha actual y la fecha de nacimiento.
fechaActual = pd.Timestamp(datetime.datetime.now())
postulantesConEdad['edad'] = (fechaActual - postulantesConEdad['fechanacimiento']).astype('<m8[Y]')

# Elimino los postulantes sin edades.
postulantesConEdad.dropna(subset = ['edad'], inplace = True)

# Descarto la columna fecha de nacimiento.
postulantesConEdad.drop('fechanacimiento', 1, inplace = True)

In [20]:
# Creo un unico dataframe para los datos del postulante.

# Renombro las columnas del dataframe de educacion a algo mas descriptivo.
postulantesEducacion.rename(columns={'nombre': 'niveleducativo', 'estado': 'estadoniveleducation'}, inplace = True)

# Hago un left join tomando todos los registros del dataframe que contiene la edad.
postulantes = pd.merge(postulantesConEdad, postulantesEducacion, on = 'idpostulante', how = 'left')

# Tomo los campos relacionado a lo educativo, los uno en uno solo y elimino las dos columnas no necesarias.
postulantes['educacion'] = postulantes['niveleducativo'] + '-' + postulantes['estadoniveleducation']
postulantes.drop('niveleducativo', 1, inplace = True)
postulantes.drop('estadoniveleducation', 1, inplace = True)

,idpostulante,sexo,edad,educacion
290853,JBr3BAk,FEM,23.0,Universitario-En Curso
46594,EMQ5J2,FEM,35.0,Posgrado-Graduado
32084,EbpE48,FEM,28.0,Secundario-Graduado
245130,ekOrQJo,FEM,27.0,Otro-Graduado
159185,kP0Q50b,FEM,23.0,Universitario-En Curso
109736,EzZODDb,FEM,30.0,Terciario/Técnico-Graduado
50065,YKq16Y,FEM,32.0,Universitario-Abandonado
209844,1QkkpKB,FEM,27.0,Otro-En Curso
291517,0zPAEWq,FEM,27.0,Universitario-Graduado
72678,6r2BEA4,FEM,33.0,Universitario-En Curso


In [6]:
# Agrego al dataframe de detalles de aviso una columna que diga especifique si sigue online o esta offline.

# Agrego una columna booleana al avisos online para que despues quede en el left merge.
avisosOnline['online'] = True

# Hago el merge entre los detalles y la tabla de online, luego completo los Nan de los offline con False.
avisos = pd.merge(avisosDetalle, avisosOnline, on = 'idaviso', how = 'left')
avisos['online'].fillna(False, inplace = True)

# Teniendo los dataframe de avisos, postulantes, vistas de aviso y postulaciones de aviso se procede a dar respuestas a las siguientes preguntas:

### 1) ¿Cuáles son las areas de trabajo mas visitadas por los diferentes grupos? ¿Y las menos visitadas?
### 2) ¿Cuáles son las areas de trabajo que reciben mas postulaciones de los diferentes grupos?

## Antes de empezar a responder las diferentes preguntas vamos a hacer un análisis demográfico de cada grupo:
- Cuantos postulantes hay en cada grupo.
- Como se conforman los diferentes grupos en cuanto a sexo.
- Cual es el nivel educacional de cada grupo y de cada grupo dividido por sexo.

In [16]:
# TODO: Gastón - Aca va el análisis demográfico.

### Antes de empezar a responder las preguntas creo los dataframe que voy a estar utilizando en el desarrollo de las mismas.

In [97]:
# Renombro la columna idAviso de las vistas a idaviso sin mayuscula así queda homogeneo.
oportunidadVistas.rename(columns={'idAviso': 'idaviso'}, inplace = True)

# Dataframe de vistas.
vistas = pd.merge(oportunidadVistas, avisos, on = 'idaviso', how = 'left')
vistas = pd.merge(vistas, postulantes, on = 'idpostulante', how = 'left')

In [103]:
# 1) ¿Cuáles son las areas de trabajo mas visitadas por los diferentes grupos? ¿Y las menos visitadas?

# Tomo del dataframe de vistas el nombre de area y la edad solamente.
areaVisitadas = vistas[['nombre_area', 'edad']]

# Agrego una columna cantidad para que luego haga las veces de contador en el group by.
areaVisitadas['cantidad'] = 1

# Divido el dataframe en los 3 grupos de edades.
areaVisitadas1 = areaVisitadas[areaVisitadas['edad'] < 23]
areaVisitadas1.drop('edad', 1, inplace = True)

areaVisitadas2 = areaVisitadas[(areaVisitadas['edad'] >= 23) & (areaVisitadas['edad'] < 35)]
areaVisitadas2.drop('edad', 1, inplace = True)

areaVisitadas3 = areaVisitadas[areaVisitadas['edad'] >= 35]
areaVisitadas3.drop('edad', 1, inplace = True)

# Agrupo segun el area.
areaVisitadas1 = areaVisitadas1.groupby(['nombre_area']).count()
areaVisitadas2 = areaVisitadas2.groupby(['nombre_area']).count()
areaVisitadas3 = areaVisitadas3.groupby(['nombre_area']).count()

# Ordeno de mayor a menor.
areaVisitadas1 = areaVisitadas1.sort_values(by='cantidad', ascending = False)
areaVisitadas2 = areaVisitadas2.sort_values(by='cantidad', ascending = False)
areaVisitadas3 = areaVisitadas3.sort_values(by='cantidad', ascending = False)

# Tomo las 10 areas mas visitadas y las menos visitadas de cada grupo.
areaMasVisitadas1 = areaVisitadas1.head(10)
areaMenosVisitadas1 = areaVisitadas1.tail(10)

areaMasVisitadas2 = areaVisitadas2.head(10)
areaMenosVisitadas2 = areaVisitadas2.tail(10)

areaMasVisitadas3 = areaVisitadas3.head(10)
areaMenosVisitadas3 = areaVisitadas3.tail(10)

,nombre_area,edad,cantidad
570744,Producción,31.0,1
951580,Call Center,27.0,1
319121,Ingeniería de Producto,28.0,1
1360875,Recepcionista,20.0,1
127834,Planeamiento comercial,52.0,1
